In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import IPython.display as display
from tqdm import tqdm
import random
import data_process

In [2]:
data_process.data_to_tfrecord(imagepath='../data/images/', outputpath='../data/face_data2.tfrecord')

이미지를 tfrecord에 저장중: 100%|██████████| 130/130 [00:00<00:00, 159.82it/s]

이미지 갯수:  130


In [56]:
filenames = '../data/face_data2.tfrecord'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset
# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),    
    'class': tf.io.FixedLenFeature([], tf.int64),    
}

def _parse_image_function(example_proto): 
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset element_spec={'class': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
for image_features in parsed_image_dataset:
    image_raw = image_features['image'].numpy()
display.display(display.Image(data=image_raw))